In [1]:
# Setup Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64
import plotly.express as px
import json
from datetime import datetime

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#  CRUD Python module file name and class name
from Artifact_One_animal_shelter_CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

## Removed MongoDB connection parameters and use CSV file instead
csv_file_path = 'aac_shelter_outcomes.csv'

# Connect to database via CRUD Module
db = AnimalShelter(csv_file_path)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# Dropping the '_id' is no longer necessary as MongoDB objectID is not going to cause problems with the CSV files anymore
# but just in case, the if statement will ensure the cleanup for safety
if '_id' in df.columns: 
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

image_filename = 'Grazioso-Salvare-Logo.png' 
try: 
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
    img_html = html.A(
    html.Img(src =f'data:image/png;base64,{encoded_image}', style ={'height': '300px'}),
        href="https://www.snhu.edu",
        target = "_blank"
    )
except FileNotFoundError:
    img_html = html.P("Logo not found.")

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Austin Animal Shelter'))),
    html.Center("Ericka Resendez Dashboard"),

    # adding data source information
    html.Div([
        html.Strong("Data Source: "), 
        html.Span(csv_file_path, style={'color': '#2980b9'}),
        html.Br() ],
          style={'textAlign': 'center', 'marginBottom': '10px'}),
    
    html.Hr(),
    html.Center(img_html), #hyperlinked logo as per instructions
    html.Hr(),

    # adding export controls section
    html.Div([
        html.H4("Data Export & Reporting", style={'color': '#2c3e50', 'textAlign': 'center'}),
        html.Div([
            html.Label("Export Format: "),
            dcc.Dropdown(
                id='export-format',
                options=[
                    {'label': 'CSV File', 'value': 'csv'},
                    {'label': 'JSON File', 'value': 'json'},
                    {'label': 'Excel File', 'value': 'excel'},
                    {'label': 'PDF Rescue Report', 'value': 'pdf'}
                ],
                value='csv',
                style={'width': '200px', 'display': 'inline-block', 'marginRight': '10px'}
            ),
            html.Button('Export Filtered Data', id='export-btn', n_clicks=0,
                       style={'background-color': '#27ae60', 'color': 'white', 'border': 'none',
                              'padding': '8px 15px', 'border-radius': '5px', 'cursor': 'pointer'}),
            html.Div(id='export-status', style={'marginTop': '10px', 'color': '#2980b9'})
        ], style={'textAlign': 'center', 'padding': '15px', 'border': '1px solid #bdc3c7', 'border-radius': '5px'})
    ]),
    
    html.Hr(),
    
    # interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(
        dcc.RadioItems(
            id = 'filter_type',
            options = [
            {'label': 'Water Rescue', 'value': 'water-btn'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'wilderness-btn'}, 
            {'label': 'Disaster or Individual Tracking', 'value' : 'disaster-btn'},
            {'label': 'Reset', 'value' : 'reset'}
        ], 
        value = 'reset', 
        inline = True        
        )
    ),
    html.Hr(),
    dash_table.DataTable(
                         id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         style_table={'overflowX': 'auto'},
                         style_data = {'whiteSpace': 'normal', 'height': 'auto'}, 
                         style_header = {'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
                         editable= False, # copied code from module six assignment
                         page_action= "native", 
                         page_current= 0,
                         page_size= 10, #limiting page to 10 rows
                         filter_action= "native", 
                         sort_action= "native",
                         sort_mode= "multi", 
                         row_selectable= 'single',
                         row_deletable = False,
                         selected_rows= [],
                         selected_columns=[]
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Div(id="query-out", style={'whiteSpace': 'pre-line'}),

    # adding hidden div to store current filter state for exports
    html.Div(id='current-filter-store', style={'display': 'none'}),
    
    # inserting a unique identifier code here.
    html.H1("Ericka's Client Server Authentication"),
])

#############################################
# Interaction Between Components / Controller
#############################################


# made adjustments to the update_dashboard section to use pandas filtering 
# instead of MongoDB query syntax as I was running into issues with the buttons
@app.callback([Output('datatable-id','data'),
              Output('datatable-id', 'columns'),
              Output('current-filter-store', 'children')],
              [Input('filter_type', 'value')])

def update_dashboard(filter_type):
    
    # Get the full dataset
    df_full = pd.DataFrame.from_records(db.read({}))
    
    if '_id' in df_full.columns:
        df_full.drop(columns=['_id'], inplace=True, errors="ignore")
    
    # Apply filters using pandas
    if filter_type == 'water-btn':
        # Water Rescue filter
        water_breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        df_filtered = df_full[
            (df_full['animal_type'] == 'Dog') &
            (df_full['breed'].isin(water_breeds)) &
            (df_full['sex_upon_outcome'] == 'Intact Female') &
            (df_full['age_upon_outcome_in_weeks'] >= 26.0) &
            (df_full['age_upon_outcome_in_weeks'] <= 156.0)
        ]
        
    elif filter_type == 'wilderness-btn':
        # Mountain or Wilderness Rescue filter
        wilderness_breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                           "Siberian Husky", "Rottweiler"]
        df_filtered = df_full[
            (df_full['animal_type'] == 'Dog') &
            (df_full['breed'].isin(wilderness_breeds)) &
            (df_full['sex_upon_outcome'] == 'Intact Male') &
            (df_full['age_upon_outcome_in_weeks'] >= 26.0) &
            (df_full['age_upon_outcome_in_weeks'] <= 156.0)
        ]
        
    elif filter_type == 'disaster-btn':
        # Disaster or Individual Tracking filter
        disaster_breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                         "Bloodhound", "Rottweiler"]
        df_filtered = df_full[
            (df_full['animal_type'] == 'Dog') &
            (df_full['breed'].isin(disaster_breeds)) &
            (df_full['sex_upon_outcome'] == 'Intact Male') &
            (df_full['age_upon_outcome_in_weeks'] >= 20.0) &
            (df_full['age_upon_outcome_in_weeks'] <= 300.0)
        ]
        
    else:  # reset or any other value
        df_filtered = df_full
    
    # Create columns definition
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
    
    # Store current filter state (simplified for CSV)
    filter_state = {"filter_type": filter_type}
    
    return df_filtered.to_dict('records'), columns, json.dumps(filter_state)
@app.callback(
        # new export functionality callback
    Output('export-status', "children"),
    [Input('export-btn', "n_clicks")],
    [State('export-format', 'value'),
     State('datatable-id', 'derived_virtual_data')]
)

def handle_export(n_clicks, export_format, current_table_data):
    """ Export functionality implementing the required multi-format export and PDF report generation """
    
    if n_clicks == 0:
        return ""
    
    try:
        if not current_table_data or len(current_table_data) == 0:
            return html.Div([
                html.Span("✗ ", style={'color': 'red', 'fontWeight': 'bold'}),
                html.Span("No data available to export")
            ])
        
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        
        # Convert current table data to DataFrame for export
        df_export = pd.DataFrame(current_table_data)
        
        # Execute the appropriate export method
        if export_format == 'csv':
            filename = f"rescue_animals_{timestamp}.csv"
            df_export.to_csv(filename, index=False)
            result = f"Data exported to {filename}"
            
        elif export_format == 'json':
            filename = f"rescue_animals_{timestamp}.json"
            df_export.to_json(filename, orient='records', indent=2)
            result = f"Data exported to {filename}"
            
        elif export_format == 'excel':
            filename = f"rescue_animals_{timestamp}.xlsx"
            df_export.to_excel(filename, index=False)
            result = f"Data exported to {filename}"
            
        elif export_format == 'pdf':
            filename = f"rescue_report_{timestamp}.pdf"
            # Use the CRUD method but pass the current data
            result = db.generate_rescue_report_pdf_from_data(current_table_data, filename)
            
        return html.Div([
            html.Span("✓ ", style={'color': 'green', 'fontWeight': 'bold'}),
            html.Span(f"Success: {result}"),
            html.Br(),
            html.Span(f"File saved as: {filename}", style={'color': '#7f8c8d', 'fontSize': '12px'})
        ])
        
    except Exception as e:
        return html.Div([
            html.Span("✗ ", style={'color': 'red', 'fontWeight': 'bold'}),
            html.Span(f"Export failed: {str(e)}")
        ])

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
        
def update_graphs(viewData):
    if viewData is None or len(viewData) ==0: 
        return [dcc.Graph(figure=px.pie(title= "No data available"))]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if 'breed' not in dff.columns or dff.empty: 
        return [dcc.Graph(figure=px.pie(title="No valid data to display"))]
    
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    fig = px.pie(data_frame = breed_counts, 
                    values = 'count', 
                    names = 'breed', 
                    color_discrete_sequence = px.colors.sequential.RdBu, 
                    title = "Breed Distribution",
                    width = 800, height= 500
                )
    
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    # Check for empty data AND handle empty selection lists
    if viewData is None or len(viewData) == 0:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[30.75, -97.48], children=[
                    dl.Tooltip("Austin, TX - Default Location"),
                    dl.Popup([
                        html.H1("Austin, Texas"),
                        html.P("Default location - No animal data selected")
                    ])
                ])
            ])
        ]
    
    dff = pd.DataFrame.from_dict(viewData)

    # handle selection 
    if index is None or len(index) == 0:
        row = 0  # Show first row if no selection
    else: 
        row = index[0]
    
    try:
        # Use column names instead of indices for safety
        if 'location_lat' in dff.columns and 'location_long' in dff.columns:
            lat = dff.iloc[row]['location_lat']
            lon = dff.iloc[row]['location_long']
        else:
            # Fallback to Austin if location columns not found
            lat, lon = 30.75, -97.48
        
        # Get animal info with safe access
        animal_name = dff.iloc[row].get('name', 'Unknown')
        animal_breed = dff.iloc[row].get('breed', 'Unknown')
        animal_type = dff.iloc[row].get('animal_type', 'Unknown')
        
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(f"{animal_type}: {animal_breed}"),
                    dl.Popup([
                        html.H1("Animal Information"),
                        html.P(f"Name: {animal_name}"),
                        html.P(f"Breed: {animal_breed}"),
                        html.P(f"Type: {animal_type}"),
                        html.P(f"Location: {lat:.4f}, {lon:.4f}")
                    ])
                ])
            ])
        ]
        
    except (IndexError, KeyError, ValueError, TypeError) as e:
        # Fallback to Austin on any error
        print(f"Map error: {e}")
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[30.75, -97.48], children=[
                    dl.Tooltip("Austin, TX - Error Fallback"),
                    dl.Popup([
                        html.H1("Austin, Texas"),
                        html.P("Showing default location due to data error")
                    ])
                ])
            ])
        ]

# run in external browser
if __name__ == '__main__':
    import webbrowser
    import threading
    import time
    
    def open_browser():
        time.sleep(3)  # Wait 3 seconds for server to start
        webbrowser.open_new('http://127.0.0.1:8050/')
    
    # Start the browser in a separate thread
    threading.Thread(target=open_browser).start()
    
    print("Starting dashboard server...")
    print("The dashboard should open automatically in your browser.")
    print("If it doesn't, please manually go to: http://127.0.0.1:8050")
    
    app.run(debug=True, host='127.0.0.1', port=8050, use_reloader=False)

Success: Loaded 10000 records from aac_shelter_outcomes.csv
Loaded 10000 records from aac_shelter_outcomes.csv
Starting dashboard server...
The dashboard should open automatically in your browser.
If it doesn't, please manually go to: http://127.0.0.1:8050
